In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
from datetime import datetime
from collections import defaultdict

In [5]:
books_engineered = pd.read_csv('data_algo/books_engineered.csv', index_col=0)

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
activities_engineered = pd.read_csv('data_algo/activities_engineered.csv', index_col=0)

In [7]:
courses_engineered = pd.read_csv('data_algo/courses_merged.csv', index_col=0)

In [8]:
books_engineered.head()

,id_issue,id_reader,id_book,author,name,reader_age,genre,is_novice,is_old,is_series,is_age_cencored,is_18_cencored
0,1,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N
1,2,179,5185.0,NaN,Сказки о солдате,30.0,poetry_folklor,N,N,Y,N,N
2,3,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N
3,4,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N
4,5,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N


In [9]:
activities_engineered.head()

,Название мероприятия,is_for_underschool,is_for_school,is_for_student,is_for_adult,is_for_retired,event_date,Semantic_tag
0,Фестивлаь современных молодежных уличных культ...,1,1,1,1,0,NaN,event_other
1,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",0,0,0,1,1,2020-05-09,time_related
2,Врасплох,0,1,1,1,1,NaN,art_drawing
3,Праздник Новогодней елки. Новогодний концерт в...,1,1,0,0,0,2020-01-01,time_related
4,"Выставка к Рождеству ""Сувенир Года""",1,1,1,1,1,2020-01-01,time_related


In [10]:
courses_engineered.head()

,status,pupil_id,service_id,Наименование_услуги,course_duration,is_newly_created,is_free,is_individual,Service_group,gender,age_cat,Onboard_year
0,3.0,25969.0,39178,Архитектура,yearly,N,Y,Y,art_drawing,M,mid_high_school,2015.0
1,3.0,158129.0,39178,Архитектура,yearly,N,Y,Y,art_drawing,F,student,2010.0
2,3.0,304476.0,39178,Архитектура,yearly,N,Y,Y,art_drawing,M,mid_high_school,2014.0
3,3.0,1139915.0,39178,Архитектура,yearly,N,Y,Y,art_drawing,M,mid_high_school,2014.0
4,3.0,298767.0,39178,Архитектура,yearly,N,Y,Y,art_drawing,F,mid_high_school,2014.0


### Readers clustering

In [11]:
books_engineered.head()

,id_issue,id_reader,id_book,author,name,reader_age,genre,is_novice,is_old,is_series,is_age_cencored,is_18_cencored
0,1,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N
1,2,179,5185.0,NaN,Сказки о солдате,30.0,poetry_folklor,N,N,Y,N,N
2,3,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N
3,4,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N
4,5,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N


In [37]:
reader_clusters = books_engineered.copy()

In [38]:
reader_clusters.shape

(11426271, 12)

In [39]:
reader_clusters = reader_clusters[reader_clusters['id_book'].notna() == True]

In [40]:
reader_clusters.shape

(11422591, 12)

In [41]:
reader_clusters['reader_book_id'] = reader_clusters['id_reader'].astype('str') + reader_clusters['id_book'].astype('str')

In [42]:
reader_clusters.tail()

,id_issue,id_reader,id_book,author,name,reader_age,genre,is_novice,is_old,is_series,is_age_cencored,is_18_cencored,reader_book_id
11426264,11423386,566211,707068.0,Даль Владимир Иванович,Пословицы и поговорки русского народа,NaN,poetry_folklor,N,Y,N,Y,N,566211707068.0
11426267,11423389,110298,394567.0,Носов Николай Николаевич,Приключения Незнайки и его друзей,62.0,russian_fiction,N,N,N,N,N,110298394567.0
11426268,11423390,110298,219667.0,NaN,Детская энциклопедия,62.0,law,N,N,N,Y,N,110298219667.0
11426269,11423391,110298,1555329.0,NaN,Веселые уроки,62.0,law,N,N,N,Y,N,1102981555329.0
11426270,11423392,535365,1069852.0,Филиппов М. А.,Патриарх Никон/М.А.Филиппов; Боярыня Морозова,62.0,russian_fiction,N,N,Y,N,N,5353651069852.0


In [43]:
reader_clusters.shape

(11422591, 13)

In [44]:
reader_clusters.drop_duplicates(['reader_book_id'], inplace=True)

In [45]:
reader_clusters.shape

(8190333, 13)

In [49]:
map_ = {'Y':1, 'N':0}

In [50]:
reader_clusters['is_novice'] = reader_clusters['is_novice'].map(map_)
reader_clusters['is_old'] = reader_clusters['is_old'].map(map_)
reader_clusters['is_series'] = reader_clusters['is_series'].map(map_)
reader_clusters['is_age_cencored'] = reader_clusters['is_age_cencored'].map(map_)
reader_clusters['is_18_cencored'] = reader_clusters['is_18_cencored'].map(map_)

In [59]:
readers_grouped = reader_clusters.groupby(['id_reader', 'genre']).agg({'genre':'count',
                                                    'is_novice':'sum',
                                                    'is_old':'sum',
                                                    'is_series':'sum',
                                                    'is_age_cencored':'sum',
                                                    'is_18_cencored':'sum',})

In [60]:
readers_grouped.rename(columns={'genre':'genre_count'}, inplace=True)

In [61]:
readers_grouped.head()

genre_count  is_novice  is_old  is_series  \
id_reader genre                                                           
163       books_philology               1          0       1          0   
          children                      1          0       0          0   
          cinema_theatre_art            2          0       1          1   
          common_scince                 2          0       0          0   
          detectives                    2          0       0          1   

                              is_age_cencored  is_18_cencored  
id_reader genre                                                
163       books_philology                   0               0  
          children                          1               0  
          cinema_theatre_art                0               0  
          common_scince                     1               0  
          detectives                        2               0

In [62]:
readers_grouped.reset_index(inplace=True)

In [63]:
readers_grouped.head()

,id_reader,genre,genre_count,is_novice,is_old,is_series,is_age_cencored,is_18_cencored
0,163,books_philology,1,0,1,0,0,0
1,163,children,1,0,0,0,1,0
2,163,cinema_theatre_art,2,0,1,1,0,0
3,163,common_scince,2,0,0,0,1,0
4,163,detectives,2,0,0,1,2,0


In [79]:
%%time
reader_profile_dict = dict()
for id_reader in readers_grouped.id_reader.unique():
    df = readers_grouped[readers_grouped['id_reader'] == id_reader]
    bagging_dict = dict()
    genres_dict = dict()
    total_books = sum(df.genre_count)
    is_novice_ratio = round(sum(df.is_novice) / total_books, 2)
    is_old_ratio = round(sum(df.is_old) / total_books, 2)
    is_series_ratio = round(sum(df.is_series) / total_books, 2)
    is_age_cencored_ratio = round(sum(df.is_age_cencored) / total_books, 2)
    is_18_cencored_ratio = round(sum(df.is_18_cencored) / total_books, 2)
    for genre in df.genre.unique():
        genre_count = int(df[df['genre'] == genre]['genre_count'])
        genre_share = round(genre_count / total_books, 2)
        genres_dict[genre] = genre_share
    bagging_dict['book_ratios'] = genres_dict
    bagging_dict['total_books'] = total_books
    bagging_dict['is_novice_ratio'] = is_novice_ratio
    bagging_dict['is_old_ratio'] = is_old_ratio
    bagging_dict['is_series_ratio'] = is_series_ratio
    bagging_dict['is_age_cencored_ratio'] = is_age_cencored_ratio
    bagging_dict['is_18_cencored_ratio'] = is_18_cencored_ratio
    reader_profile_dict[id_reader] = bagging_dict

Wall time: 33min 31s


In [ ]:
############################################################################################################################

In [77]:
reader_profile_dict[163]['book_ratios']['books_philology']

0.02

In [78]:
reader_profile_dict

{163: {'book_ratios': {'books_philology': 0.02,
   'children': 0.02,
   'cinema_theatre_art': 0.05,
   'common_scince': 0.05,
   'detectives': 0.05,
   'fantastics_fantasy': 0.05,
   'history_hronicles': 0.17,
   'law': 0.05,
   'russian_fiction': 0.48,
   'socio_human': 0.05,
   'technical_sciense': 0.02},
  'total_books': 42,
  'is_novice_ratio': 0.0,
  'is_old_ratio': 0.45,
  'is_series_ratio': 0.29,
  'is_age_cencored_ratio': 0.17,
  'is_18_cencored_ratio': 0.0},
 164: {'book_ratios': {'history_hronicles': 1.0},
  'total_books': 1,
  'is_novice_ratio': 0.0,
  'is_old_ratio': 0.0,
  'is_series_ratio': 1.0,
  'is_age_cencored_ratio': 0.0,
  'is_18_cencored_ratio': 0.0}}

In [ ]:
# find the last taken book and age

In [80]:
readers_last_book = reader_clusters[['id_reader', 'id_book', 'reader_age']]

In [90]:
readers_last_book.drop_duplicates(['id_reader'], keep='last', inplace=True)

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
last_book_dict = dict()
age_dict = dict()
for row in readers_last_book.itertuples():
    last_book_dict[row[1]] = row[2]
    age_dict[row[1]] = row[3]

In [94]:
last_book_dict

{179: 5185.0,
 246: 13103.0,
 257: 1164.0,
 455: 22341.0,
 531: 24895.0,
 546: 1862.0,
 556: 440486.0,
 602: 25726.0,
 617: 1292388.0,
 616: 365659.0,
 660: 659756.0,
 666: 26485.0,
 664: 26476.0,
 670: 26572.0,
 695: 353826.0,
 723: 26561.0,
 835: 292346.0,
 858: 32695.0,
 863: 10379.0,
 899: 7857.0,
 914: 9636.0,
 430: 36339.0,
 929: 717.0,
 940: 36610.0,
 980: 31425.0,
 989: 33847.0,
 1003: 4169.0,
 1136: 16388.0,
 1236: 20026.0,
 1272: 32004.0,
 1278: 35908.0,
 1281: 48602.0,
 637: 13032.0,
 1337: 42459.0,
 1299: 22186.0,
 726: 49489.0,
 1133: 16871.0,
 1314: 336858.0,
 1409: 1422246.0,
 1410: 419285.0,
 891: 152809.0,
 1496: 42251.0,
 1497: 36889.0,
 1547: 602355.0,
 1620: 293770.0,
 1365: 15204.0,
 1720: 634624.0,
 1783: 47355.0,
 1788: 28466.0,
 655: 14969.0,
 486: 6308.0,
 555: 218958.0,
 1989: 12520.0,
 2017: 532401.0,
 2043: 24189.0,
 1246: 7232.0,
 928: 1618339.0,
 2167: 1409291.0,
 2164: 7913.0,
 683: 600.0,
 1285: 30752.0,
 1286: 50677.0,
 2190: 28631.0,
 1239: 43172.0,
 2

In [96]:
%%time
for key in reader_profile_dict.keys():
    last_book = last_book_dict[key]
    age = age_dict[key]
    reader_profile_dict[key]['last_book'] = last_book
    reader_profile_dict[key]['age'] = age

Wall time: 521 ms


In [97]:
test_key = list(reader_profile_dict.keys())[0]
reader_profile_dict[test_key]

{'book_ratios': {'books_philology': 0.02,
  'children': 0.02,
  'cinema_theatre_art': 0.05,
  'common_scince': 0.05,
  'detectives': 0.05,
  'fantastics_fantasy': 0.05,
  'history_hronicles': 0.17,
  'law': 0.05,
  'russian_fiction': 0.48,
  'socio_human': 0.05,
  'technical_sciense': 0.02},
 'total_books': 42,
 'is_novice_ratio': 0.0,
 'is_old_ratio': 0.45,
 'is_series_ratio': 0.29,
 'is_age_cencored_ratio': 0.17,
 'is_18_cencored_ratio': 0.0,
 'last_book': 841833.0,
 'age': 38.0}

In [98]:
import pickle

In [100]:
with open('important_backlog/readers_clusters.pickle', 'wb') as f:
    pickle.dump(reader_profile_dict, f)

In [ ]:
# calculate average ratios

In [101]:
%%time
total_books = []
is_novice_ratio = []
is_old_ratio = []
is_series_ratio = []
is_age_cencored_ratio = []
is_18_cencored_ratio = []
for key in reader_profile_dict.keys():
    total_books.append(reader_profile_dict[key]['total_books'])
    is_novice_ratio.append(reader_profile_dict[key]['is_novice_ratio'])
    is_old_ratio.append(reader_profile_dict[key]['is_old_ratio'])
    is_series_ratio.append(reader_profile_dict[key]['is_series_ratio'])
    is_age_cencored_ratio.append(reader_profile_dict[key]['is_age_cencored_ratio'])
    is_18_cencored_ratio.append(reader_profile_dict[key]['is_18_cencored_ratio'])

Wall time: 446 ms


In [104]:
from statistics import median

In [105]:
median(total_books)

8

In [106]:
median_ratios_dict = dict()
median_ratios_dict['total_books'] = median(total_books)
median_ratios_dict['is_novice_ratio'] = median(is_novice_ratio)
median_ratios_dict['is_old_ratio'] = median(is_old_ratio)
median_ratios_dict['is_series_ratio'] = median(is_series_ratio)
median_ratios_dict['is_age_cencored_ratio'] = median(is_age_cencored_ratio)
median_ratios_dict['is_18_cencored_ratio'] = median(is_18_cencored_ratio)

In [107]:
with open('important_backlog/average_readers_ratios.pickle', 'wb') as f:
    pickle.dump(median_ratios_dict, f)

### Books clustering

In [108]:
books_engineered.head()

,id_issue,id_reader,id_book,author,name,reader_age,genre,is_novice,is_old,is_series,is_age_cencored,is_18_cencored
0,1,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N
1,2,179,5185.0,NaN,Сказки о солдате,30.0,poetry_folklor,N,N,Y,N,N
2,3,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N
3,4,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N
4,5,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N


In [176]:
from collections import defaultdict

In [177]:
books_clusters = defaultdict(list)

In [178]:
for row in books_engineered.itertuples():
    book_profile_dict = dict()
    book_profile_dict['book_id'] = row[3]
    book_profile_dict['author'] = row[4]
    book_profile_dict['title'] = row[5]
    book_profile_dict['is_novice'] = row[8]
    book_profile_dict['is_old'] = row[9]
    book_profile_dict['is_series'] = row[10]
    book_profile_dict['is_age_cencored'] = row[11]
    book_profile_dict['is_18_cencored'] = row[12]
    books_clusters[row[7]].append(book_profile_dict)

In [179]:
books_clusters.keys()

dict_keys(['russian_fiction', 'poetry_folklor', 'history_hronicles', 'books_philology', 'detectives', 'psychology', 'socio_human', 'common_scince', 'philosophy_religion_culture', 'cinema_theatre_art', 'pedagogical', 'biography', 'law', 'technical_sciense', 'children', 'educational_school', 'medicine', 'hobbies_home_other', 'foreign_fiction', 'fantastics_fantasy', 'economics_business', nan, 'love_sentiment', 'humour_cites', 'countries_journeys', 'it', 'politics', 'foreign_lang', 'journals_periodics', 'horrors'])

In [180]:
len(books_clusters['russian_fiction'])

5359297

In [181]:
with open('important_backlog/books_clusters.pickle', 'wb') as f:
    pickle.dump(books_clusters, f)

### Courses clustering

In [182]:
courses_engineered.head()

,status,pupil_id,service_id,Наименование_услуги,course_duration,is_newly_created,is_free,is_individual,Service_group,gender,age_cat,Onboard_year
0,3.0,25969.0,39178,Архитектура,yearly,0,1,1,art_drawing,M,mid_high_school,2015.0
1,3.0,158129.0,39178,Архитектура,yearly,0,1,1,art_drawing,F,student,2010.0
2,3.0,304476.0,39178,Архитектура,yearly,0,1,1,art_drawing,M,mid_high_school,2014.0
3,3.0,1139915.0,39178,Архитектура,yearly,0,1,1,art_drawing,M,mid_high_school,2014.0
4,3.0,298767.0,39178,Архитектура,yearly,0,1,1,art_drawing,F,mid_high_school,2014.0


In [183]:
courses_clusters = defaultdict(list)

In [184]:
for row in courses_engineered.itertuples():
    course_profile_dict = dict()
    course_profile_dict['course_id'] = row[3]
    course_profile_dict['title'] = row[4]
    course_profile_dict['course_duration'] = row[5]
    course_profile_dict['is_new'] = row[6]
    course_profile_dict['is_free'] = row[7]
    course_profile_dict['is_individual'] = row[8]
    course_profile_dict['onboarding_uear'] = row[12]
    courses_clusters[row[9]].append(course_profile_dict)

In [185]:
courses_clusters.keys()

dict_keys(['art_drawing', 'add_edu', 'music_singing_vocal', 'dances_theatre', 'music_instruments', 'theatre_cinema', 'hobbies', 'hobbies_craftery', 'history', nan, 'games', 'languages', 'sport_fitness'])

In [186]:
len(courses_clusters['art_drawing'])

65070

In [187]:
with open('important_backlog/courses_clusters.pickle', 'wb') as f:
    pickle.dump(courses_clusters, f)

### Activities clustering

In [188]:
activities_engineered.head()

,Название мероприятия,is_for_underschool,is_for_school,is_for_student,is_for_adult,is_for_retired,event_date,Semantic_tag
0,Фестивлаь современных молодежных уличных культ...,1,1,1,1,0,NaN,event_other
1,"Танцевальный вечер ""Ретро"" в честь Дня Победы ...",0,0,0,1,1,2020-05-09,time_related
2,Врасплох,0,1,1,1,1,NaN,art_drawing
3,Праздник Новогодней елки. Новогодний концерт в...,1,1,0,0,0,2020-01-01,time_related
4,"Выставка к Рождеству ""Сувенир Года""",1,1,1,1,1,2020-01-01,time_related


In [189]:
activities_clusters = defaultdict(list)

In [190]:
for row in activities_engineered.itertuples():
    activity_profile_dict = dict()
    activity_profile_dict['title'] = row[1]
    activity_profile_dict['is_for_underschool'] = row[2]
    activity_profile_dict['is_for_school'] = row[3]
    activity_profile_dict['is_for_student'] = row[4]
    activity_profile_dict['is_for_adult'] = row[5]
    activity_profile_dict['is_for_retired'] = row[6]
    activity_profile_dict['event_date'] = row[7]
    activities_clusters[row[8]].append(activity_profile_dict)

In [195]:
activities_clusters

defaultdict(list,
            {'event_other': [{'title': 'Фестивлаь современных молодежных уличных культур "Вызов Принят"',
               'is_for_underschool': 1,
               'is_for_school': 1,
               'is_for_student': 1,
               'is_for_adult': 1,
               'is_for_retired': 0,
               'event_date': nan},
              {'title': 'Флексагон',
               'is_for_underschool': 1,
               'is_for_school': 1,
               'is_for_student': 1,
               'is_for_adult': 0,
               'is_for_retired': 0,
               'event_date': nan},
              {'title': 'Участие в  Московском пленэре.',
               'is_for_underschool': 1,
               'is_for_school': 1,
               'is_for_student': 1,
               'is_for_adult': 1,
               'is_for_retired': 1,
               'event_date': nan},
              {'title': 'Игрушки из солёного теста',
               'is_for_underschool': 1,
               'is_for_school': 1,
     

In [191]:
activities_clusters.keys()

dict_keys(['event_other', 'time_related', 'art_drawing', 'place_related', 'sport_competition_game', 'foreign_languages', 'dance', nan, 'cinema_theatre', 'discussions_forums', 'education', 'biography', 'history', 'music_instruments', 'folklor', 'music'])

In [196]:
len(activities_clusters['discussions_forums'])

3355

In [197]:
with open('important_backlog/activities_clusters.pickle', 'wb') as f:
    pickle.dump(activities_clusters, f)

### Pupils clustering

In [109]:
courses_engineered.head()

,status,pupil_id,service_id,Наименование_услуги,course_duration,is_newly_created,is_free,is_individual,Service_group,gender,age_cat,Onboard_year
0,3.0,25969.0,39178,Архитектура,yearly,N,Y,Y,art_drawing,M,mid_high_school,2015.0
1,3.0,158129.0,39178,Архитектура,yearly,N,Y,Y,art_drawing,F,student,2010.0
2,3.0,304476.0,39178,Архитектура,yearly,N,Y,Y,art_drawing,M,mid_high_school,2014.0
3,3.0,1139915.0,39178,Архитектура,yearly,N,Y,Y,art_drawing,M,mid_high_school,2014.0
4,3.0,298767.0,39178,Архитектура,yearly,N,Y,Y,art_drawing,F,mid_high_school,2014.0


In [111]:
courses_engineered['is_newly_created'] = courses_engineered['is_newly_created'].map(map_)
courses_engineered['is_free'] = courses_engineered['is_free'].map(map_)
courses_engineered['is_individual'] = courses_engineered['is_individual'].map(map_)

In [120]:
pupils_df = courses_engineered.copy()

In [121]:
dummy_duration = pd.get_dummies(pupils_df['course_duration'])

In [122]:
pupils_df = pd.concat([pupils_df, dummy_duration], axis=1)

In [123]:
pupils_df.fillna(0, inplace=True)

In [124]:
pupils_df.head()

,status,pupil_id,service_id,Наименование_услуги,course_duration,is_newly_created,is_free,is_individual,Service_group,gender,age_cat,Onboard_year,daily,monthly,yearly
0,3.0,25969.0,39178,Архитектура,yearly,0,1,1,art_drawing,M,mid_high_school,2015.0,0,0,1
1,3.0,158129.0,39178,Архитектура,yearly,0,1,1,art_drawing,F,student,2010.0,0,0,1
2,3.0,304476.0,39178,Архитектура,yearly,0,1,1,art_drawing,M,mid_high_school,2014.0,0,0,1
3,3.0,1139915.0,39178,Архитектура,yearly,0,1,1,art_drawing,M,mid_high_school,2014.0,0,0,1
4,3.0,298767.0,39178,Архитектура,yearly,0,1,1,art_drawing,F,mid_high_school,2014.0,0,0,1


In [130]:
pupils_df.isnull().sum()

status                 0
pupil_id               0
service_id             0
Наименование_услуги    0
course_duration        0
is_newly_created       0
is_free                0
is_individual          0
Service_group          0
gender                 0
age_cat                0
Onboard_year           0
daily                  0
monthly                0
yearly                 0
dtype: int64

In [131]:
pupils_df[pupils_df.pupil_id == 0].shape

(77036, 15)

In [132]:
pupils_df = pupils_df[pupils_df.pupil_id != 0]

In [133]:
pupils_grouped = pupils_df.groupby(['pupil_id', 'Service_group']).agg({'Service_group':'count',
                                                    'is_newly_created':'sum',
                                                    'is_free':'sum',
                                                    'is_individual':'sum',
                                                    'daily':'sum',
                                                    'monthly':'sum',
                                                    'yearly':'sum'})

In [134]:
pupils_grouped.head()

,,Service_group,is_newly_created,is_free,is_individual,daily,monthly,yearly
pupil_id,Service_group,,,,,,,
1.0,music_singing_vocal,5,0,5,1,0,0,5
2.0,music_singing_vocal,5,0,5,1,0,0,5
3.0,music_singing_vocal,6,0,6,2,0,0,6
4.0,music_singing_vocal,1,0,1,1,0,0,1
5.0,music_singing_vocal,5,0,5,1,0,0,5


In [135]:
pupils_grouped.rename(columns={'Service_group':'service_group_count'}, inplace=True)

In [136]:
pupils_grouped.reset_index(inplace=True)

In [139]:
pupils_grouped.head()

,pupil_id,Service_group,service_group_count,is_newly_created,is_free,is_individual,daily,monthly,yearly
0,1.0,music_singing_vocal,5,0,5,1,0,0,5
1,2.0,music_singing_vocal,5,0,5,1,0,0,5
2,3.0,music_singing_vocal,6,0,6,2,0,0,6
3,4.0,music_singing_vocal,1,0,1,1,0,0,1
4,5.0,music_singing_vocal,5,0,5,1,0,0,5


In [160]:
%%time
pupil_profile_dict = dict()
for pupil_id in pupils_grouped.pupil_id.unique():
    df = pupils_grouped[pupils_grouped['pupil_id'] == pupil_id]
    bagging_dict = dict()
    courses_dict = dict()
    total_courses = sum(df.service_group_count)
    is_free_ratio = round(sum(df.is_free) / total_courses, 2)
    is_individual_ratio = round(sum(df.is_individual) / total_courses, 2)
    daily_ratio = round(sum(df.daily) / total_courses, 2)
    monthly_ratio = round(sum(df.monthly) / total_courses, 2)
    yearly_ratio = round(sum(df.yearly) / total_courses, 2)
    for course in df.Service_group.unique():
        course_count = int(df[df['Service_group'] == course]['service_group_count'])
        course_share = round(course_count / total_courses, 2)
        courses_dict[course] = course_share
    bagging_dict['course_ratios'] = courses_dict
    bagging_dict['total_courses'] = total_courses
    bagging_dict['is_free_ratio'] = is_free_ratio
    bagging_dict['is_individual_ratio'] = is_individual_ratio
    bagging_dict['daily_ratio'] = daily_ratio
    bagging_dict['monthly_ratio'] = monthly_ratio
    bagging_dict['yearly_ratio'] = yearly_ratio
    pupil_profile_dict[pupil_id] = bagging_dict

Wall time: 9min 43s


In [ ]:
#here##############################################################################################################

In [161]:
gender_age = pupils_df[['pupil_id', 'gender', 'age_cat', 'service_id']]

In [162]:
gender_age.drop_duplicates(['pupil_id'], keep='last', inplace=True)

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [163]:
gender_age.head(50)

,pupil_id,gender,age_cat,service_id
129,908331.0,F,mid_high_school,39178
168,1145975.0,F,mid_high_school,39178
247,970738.0,M,mid_high_school,39178
317,1169368.0,M,mid_high_school,39178
394,22949.0,M,mid_high_school,87877
400,22954.0,M,mid_high_school,87877
415,22964.0,F,mid_high_school,87877
419,22968.0,M,young_professionals,87877
423,22971.0,M,mid_high_school,87877
429,22972.0,M,mid_high_school,87877


In [164]:
gender_dict = dict()
age_cat = dict()
last_dict = dict()
for row in gender_age.itertuples():
    gender_dict[row[1]] = row[2]
    age_cat[row[1]] = row[3]
    last_dict[row[1]] = row[4] 

In [165]:
age_cat[1]

'mid_high_school'

In [166]:
pupil_profile_dict[1]

{'course_ratios': {'music_singing_vocal': 1.0},
 'total_courses': 5,
 'is_free_ratio': 1.0,
 'is_individual_ratio': 0.2,
 'daily_ratio': 0.0,
 'monthly_ratio': 0.0,
 'yearly_ratio': 1.0}

In [169]:
%%time
errors = []
for key in pupil_profile_dict.keys():
    gender = gender_dict[key]
    try:
        age_cat = age_cat[key]
    except TypeError:
        age_cat = 'undefined'
    last = last_dict[key]
    pupil_profile_dict[key]['gender'] = gender
    pupil_profile_dict[key]['age_cat'] = age_cat
    pupil_profile_dict[key]['last_service'] = last

Wall time: 412 ms


In [170]:
test_key = list(pupil_profile_dict.keys())[0]
pupil_profile_dict[test_key]

{'course_ratios': {'music_singing_vocal': 1.0},
 'total_courses': 5,
 'is_free_ratio': 1.0,
 'is_individual_ratio': 0.2,
 'daily_ratio': 0.0,
 'monthly_ratio': 0.0,
 'yearly_ratio': 1.0,
 'gender': 'F',
 'age_cat': 'undefined',
 'last_service': 144984}

In [171]:
with open('important_backlog/pupils_clusters.pickle', 'wb') as f:
    pickle.dump(pupil_profile_dict, f)

In [172]:
%%time
total_courses = []
is_free_ratio = []
is_individual_ratio = []
daily_ratio = []
monthly_ratio = []
yearly_ratio = []
for key in pupil_profile_dict.keys():
    total_courses.append(pupil_profile_dict[key]['total_courses'])
    is_free_ratio.append(pupil_profile_dict[key]['is_free_ratio'])
    is_individual_ratio.append(pupil_profile_dict[key]['is_individual_ratio'])
    daily_ratio.append(pupil_profile_dict[key]['daily_ratio'])
    monthly_ratio.append(pupil_profile_dict[key]['monthly_ratio'])
    yearly_ratio.append(pupil_profile_dict[key]['yearly_ratio'])

Wall time: 313 ms


In [173]:
median(total_courses)

2

In [174]:
median_ratios_dict = dict()
median_ratios_dict['total_courses'] = median(total_courses)
median_ratios_dict['is_free_ratio'] = median(is_free_ratio)
median_ratios_dict['is_individual_ratio'] = median(is_individual_ratio)
median_ratios_dict['daily_ratio'] = median(daily_ratio)
median_ratios_dict['monthly_ratio'] = median(monthly_ratio)
median_ratios_dict['yearly_ratio'] = median(yearly_ratio)

In [175]:
with open('important_backlog/average_pupils_ratios.pickle', 'wb') as f:
    pickle.dump(median_ratios_dict, f)